### 볼린저 밴드 
1. 주식 데이터 로드 
2. 결측치와 이상치를 제거 
3. 시간, 종가로 이루어진 데이터프레임 변경
4. 이동 평균선 : 데이터의 20개의 평균을 구해서 새로운 파생변수 대입
5. 상단 밴드 : 이동 평균선 + (2 * 데이터 20개의 표준편차)
6. 하단 밴드 : 이동 평균전 - (2 * 데이터 20개의 표준편차)
7. 구매 상태를 확인하는 파생변수 생성
8. 구매 상태를 입력
9. 수익율 계산

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../../csv/AMZN.csv", index_col='Date')
df.head(1)

In [ ]:
# 데이터프레임에서 수정종가만 있는 데이터프레임으로 변환
price_df = df[['Adj Close']]

In [ ]:
# 결측치, 무한대값 제거 
# ~ 연산자 : True와 False 반대로 표시 
flag = ~price_df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
price_df = price_df.loc[flag]

In [ ]:
# 이동 평균선 생성 
# 종가 데이터의 20개의 평균
price_df['center'] = 0
for i in range(20, len(price_df)):
    # 반복문에서 i는 ? -> 데이터프레임의 인덱스 위치
    # 20개 데이터의 평균을 구한다.
    center_data = price_df.iloc[i-20:i-1, 0].mean()
    price_df.iloc[i, 1] = center_data
price_df.iloc[19:25]

In [ ]:
# 이동 평균선 : 20개의 데이터의 평균
price_df['center'] = price_df['Adj Close'].rolling(20).mean()

In [ ]:
# 상단 밴드, 하단 밴드 
# 상단밴드 
price_df['ub'] = price_df['center'] + (2 * price_df['Adj Close'].rolling(20).std())
# 하단밴드 
price_df['lb'] = price_df['center'] - (2 * price_df['Adj Close'].rolling(20).std())

In [ ]:
price_df.iloc[19:22]

In [ ]:
price_df.tail(100).drop('Adj Close', axis=1).plot()

In [ ]:
# 인덱스를 시계열로 변경 
price_df.index  = pd.to_datetime(price_df.index, format='%Y-%m-%d')

In [ ]:
from datetime import datetime

In [ ]:
start = '20100101'

start = datetime.strptime(start, '%Y%m%d').isoformat()

test_df = price_df.loc[start:]

In [ ]:
test_df.head(2)

In [ ]:
# 거래내역 파생변수 하나 생성
test_df['trade'] = ""

In [ ]:
test_df.head(2)

### 구매 내역 추가 
- 조건 
    - 상단밴드보다 종가가 높은 경우
        - 현재 구매 상태이면 
            - 매도 (trade="")
        - 현재 구매 상태가 아니면
            - 아무 행동도 하지 않는다 (trade="")
    - 하단밴드보다 종가가 낮은 경우
        - 현재 구매 상태이면
            - 아무 행동도 하지 않는다. (trade='buy')
        - 현재 구매 상태가 아니면 
            - 매수 (trade='buy')
    - 종가가 하단밴드와 상단밴드 사이에 있는 경우
        - 현재 구매 상태이면 
            - 아무 행동도 하지 않는다 (trade='buy')
        - 현재 구매 상태가 아니면 
            - 아무 행동도 하지 않는다. (trade='')

In [ ]:
for i in test_df.index:
    # i는 ? -> test_df.index 값들이 하나씩 대입하여 반복
    
    # 상단 밴드보다 종가가 높은 경우
    if test_df.loc[i, 'Adj Close'] > test_df.loc[i, 'ub']:
        # 현재 구매상태인지 ?
        if test_df.shift(1).loc[i, 'trade'] == 'buy':
            test_df.loc[i, 'trade'] = ''
        else:
            test_df.loc[i, 'trade'] = ''
    # 하단 밴드보다 종가가 낮은 경우
    elif test_df.loc[i, 'Adj Close'] < test_df.loc[i, 'lb']:
        # 현재 구매상태라면?
        if test_df.shift(1).loc[i, 'trade'] == 'buy':
            test_df.loc[i, 'trade'] = 'buy'
        else:
            test_df.loc[i, 'trade'] = 'buy'
    # 밴드 사이에 종가가 존재하는 경우?
    else:
        # 현재 구매상태라면?
        if test_df.shift(1).loc[i, 'trade'] == 'buy':
            test_df.loc[i, 'trade'] = 'buy'
        else:
            test_df.loc[i, 'trade'] = ''

In [ ]:
test_df['trade'].value_counts()

### 수익율 계산
- 구매한 날의 종가
    - 전날의 trade가 "" 오늘의 trade가 'buy' 인 날짜의 종가 
- 판매한 날의 종가
    - 전날의 trade가 'buy' 이고 오늘의 trade가 ''인 날자의 종가
- 수익율 계산
    - (판매 가격 - 구매 가격) / 구매 가격  + 1
- 구매 가격과 판매 가격을 초기화
- 여러개의 수익율이 생성
- 수익율을 누적 곱 -> 누적 수익율


In [ ]:
rtn = 1.0
buy = 0.0
sell = 0.0

# 수익율 파생변수 생성
test_df['return'] = 1

for i in test_df.index:
    # 구매가를 대입 
    if (test_df.shift(1).loc[i, 'trade'] == '') and \
        (test_df.loc[i, 'trade'] == 'buy'):
        buy = test_df.loc[i, 'Adj Close']
        print('매수 일 :', i, '매수가 :', buy)
    # 판매가를 대입
    elif (test_df.shift(1).loc[i, 'trade'] == 'buy') and \
        (test_df.loc[i, 'trade'] == ''):
        sell = test_df.loc[i, 'Adj Close']
        # 수익율 계산
        rtn = (sell - buy) / buy + 1
        test_df.loc[i, 'return'] = rtn
        print('매도 일 :', i, "매도가 :", sell, "수익율 :", rtn)

    # 구매가, 판매가 초기화
    if test_df.loc[i, 'trade'] == '' :
        buy = 0.0
        sell = 0.0

In [ ]:
# 누적 수익율 계산
acc_rtn = 1.0

for i in test_df.index:
    rtn = test_df.loc[i, 'return']
    # acc_rtn = acc_rtn * rtn
    acc_rtn *= rtn
    test_df.loc[i, 'acc_rtn'] = acc_rtn

print('누적 수익율 :', acc_rtn)

### 볼린저밴드 함수화
1. 첫 번째 함수 매개변수 4개(데이터프레임, 선택 컬럼, 시작시간, 종료시간)
    - 인덱스가 Date인지 확인 후 아니라면 인덱스를 Date로 변경
    - 인덱스를 시계열로 변경
    - 선택 컬럼만 두고 나머지 컬럼들을 모두 삭제  
    - 결측치, 이상치(무한대) 값들을 제거 
    - 이동평균선, 상단밴드, 하단밴드 파생변수 생성
    - 시작시간과 종료시간으로 데이터를 필터링
    - 결과를 리턴
2. 두 번째 함수 생성 매개변수 1개(데이터프레임(1번 함수에서 나온 결과), 선택 컬럼)
    - 거래 내역이라는 파생 변수 생성 (데이터는 "")
    - 볼린저 밴드를 이용하여 거래 내역 추가&출력(print)
    - 결과를 리턴
3. 세 번째 함수 생성 매개 변수 1개 (데이터프레임(2번 함수에서 나온 결과))
    - 수익율 파생 변수 생성 (데이터는 1)
    - 판매를 한 날의 수익율 변경
    - 누적 수익율을 계산하여 새로운 파생변수에 대입 
    - 최종 수익율을 출력(print)
    - 결과를 리턴

In [ ]:
# 첫번째 함수 생성
def create_band(df, col = 'Close', start = '20100101', end = '20230818'):
    # 컬럼에 Date가 존재한다면
    if 'Date' in df.columns:
        # Date를 인덱스로 변경
        df = df.set_index('Date')
    # 인덱스를 시계열 변경
    df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
    # 선택한 컬럼을 제외한 나머지 컬럼은 삭제 -> 선택한 컬럼만 출력
    df = df[[col]]
    # 결측치, 무한대가 아닌 인덱스의 조건식
    flag = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    # 인덱스 조건식을 df에 넣어준다. 
    df = df.loc[flag]
    # 이동 평균선 생성
    df['center'] = df[col].rolling(20).mean()
    # 상단 밴드 생성
    df['ub'] = df['center'] + (2 * df[col].rolling(20).std())
    # 하단 밴드 생성
    df['lb'] = df['center'] - (2 * df[col].rolling(20).std())
    # 시작 시간과 종료시간으로 필터링
    start = datetime.strptime(start, '%Y%m%d').isoformat()
    end = datetime.strptime(end, '%Y%m%d').isoformat()
    df = df.loc[start:end]
    return df

In [ ]:
df2 = pd.read_csv('../../csv/BND.csv')

In [ ]:
result = create_band(df2, 'Close', '20100101', '20230818')

In [ ]:
# 두 번째 함수 생성
# 매개변수 2개(데이터프레임, 컬럼)
def add_trade(df):
    col = df.columns[0]
    # 거래 내역이라는 파생 변수를 생성
    df['trade'] = ""
    # 거래 내역을 추가
    for i in df.index:
        # 상단밴드보다 종가가 높은 경우
        if df.loc[i, col] > df.loc[i, 'ub']:
            df.loc[i, 'trade'] = ''
        # 종가가 하단밴드보다 낮은 경우
        elif df.loc[i, col] < df.loc[i, 'lb']:
            df.loc[i, 'trade'] = 'buy'
            # 구매 날짜에 출력
            if df.shift(1).loc[i, 'trade'] == '':
                print('구매일 :', i)
        # 종가가 밴드 사이에 있는 경우
        else:
            # 현재 구매상태라면?
            if df.shift(1).loc[i, 'trade'] == 'buy':
                df.loc[i, 'trade'] = 'buy'
            else:
                df.loc[i, 'trade'] = ''
    
    return df
        

In [ ]:
result = add_trade(result)

In [ ]:
# 세 번째 함수
def add_rtn(df):
    # 기준이 되는 종가
    col = df.columns[0]
    # 수익율 파생변수 생성(데이터 1)
    df['return'] = 1
    # 수익율, 구매가, 판매가 변수 생성
    rtn = 1.0
    buy = 0.0
    sell = 0.0

    # 수익율 대입 
    for i in df.index:
        # 매수가를 입력
        if (df.shift(1).loc[i, 'trade'] == '') and \
            (df.loc[i, 'trade'] == 'buy'):
            buy = df.loc[i, col]
            print('매수일 :', i, '매수가 :', buy)
        # 매도가, 수익율을 입력
        elif (df.shift(1).loc[i, 'trade'] == 'buy') and \
            (df.loc[i, 'trade'] == ''):
            sell = df.loc[i, col]
            # 수익율 계산
            # buy -> 100, sell -> 120  수익율 +20% / 120%
            # (120 - 100) / 100 + 1 -> 1.2
            rtn = (sell - buy) / buy + 1
            # 수익율을 데이터프레임에 대입
            df.loc[i, 'return'] = rtn
            print('매도일 :', i, '매도가 :', sell, '수익율 :', rtn)
        
    # 누적 수익율 계산

    # 누적 수익율 기본값 생성
    acc_rtn = 1.0

    for i in df.index:
        rtn = df.loc[i, 'return']
        acc_rtn *= rtn
        df.loc[i, 'acc_rtn'] = acc_rtn
    
    # 누적 수익율을 출력
    print("누적 수익율 :",acc_rtn)

    return df

In [ ]:
add_rtn(result)

In [ ]:
result['trade'].value_counts()

In [ ]:
import bollinger

In [ ]:
df = pd.read_csv('../../csv/AAPL.csv')

In [ ]:
# 밴드 생성 함수 호출
result = bollinger.create_band(df, start = '20130101')

In [ ]:
result.head(3)

In [ ]:
# 거래 내역 추가 함수 호출
result = bollinger.add_trade(result)
result.head(3)

In [ ]:
# 수익율 추가, 수익율 계산 하는 함수 호출
result = bollinger.add_rtn(result)
result.tail(3)